In [ ]:
# bibliotecas
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import numpy as np
import torch

from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models

from sklearn.metrics import classification_report, confusion_matrix, f1_score, roc_auc_score
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split

from torch.utils.data import Subset, DataLoader, ConcatDataset, random_split
from collections import Counter

### Carregamento e Pré-Processamento do Dataset

In [ ]:
from utils import data_utils

image_size = (224, 224)
train_data, test_data = data_utils.load_and_preprocess_data(resize_to=image_size,
                                                            keras_format=True,
                                                            model_type=data_utils.ModelType.EFFICIENTNET_B0)

### Divisão Holdout

In [ ]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
train_idx, val_idx = next(split.split(np.zeros(len(train_data.targets)), train_data.targets))


In [ ]:
train_split = Subset(train_data, train_idx)
val_split = Subset(train_data, val_idx)

In [ ]:
print(f"Treino:    {len(train_split)} imagens")
print(f"Validação: {len(val_split)} imagens")
print(f"Teste:     {len(test_data)} imagens")

In [ ]:
def get_X_y(dataset):
    X = []
    y = []
    for img, label in dataset:
        X.append(img.numpy())  
        y.append(label)
    X = np.stack(X)
    y = np.array(y)
    return X, y


In [ ]:
X_train, y_train = get_X_y(train_split)
X_val, y_val = get_X_y(val_split)
X_test, y_test = get_X_y(test_data)

## Treino

### Modelo

**EfficientNetB0**

EfficientNet é uma família de CNNs que visa alcançar alto desempenho com menos recursos computacionais em comparação com arquiteturas anteriores. Foi apresentada por Mingxing Tan e Quoc V. Le, do Google Research, em seu artigo de 2019 "EfficientNet: Repensando o Escalonamento de Modelos para Redes Neurais Convolucionais". A ideia central por trás do EfficientNet é um novo método de escalonamento que escalona uniformemente todas as dimensões de profundidade, largura e resolução usando um coeficiente composto.

O EfficientNet utiliza uma técnica chamada coeficiente composto para escalonar modelos de maneira simples, mas eficaz. Em vez de escalonar aleatoriamente a largura, a profundidade ou a resolução, o escalonamento composto escalona uniformemente cada dimensão com um conjunto fixo de coeficientes de escalonamento. Usando esse método de escalonamento e o AutoML, os autores do EfficientNet desenvolveram sete modelos de várias dimensões, que superaram a precisão de ponta da maioria das CNNs e com eficiência muito superior.

A rede EfficientNet-B0 é composta por três partes principais: o Stem, o Body e o Head.

O Stem é a camada inicial da rede, composta por uma convolução padrão com 32 filtros, tamanho de kernel 3x3 e stride 2, seguida por uma normalização em batch (batch normalization) e uma função de ativação ReLU6. O Body consiste em uma série de blocos MBConv com diferentes configurações. Esses blocos utilizam convoluções depthwise separáveis, que reduzem a complexidade computacional, e camadas squeeze-and-excitation, que ajudam a rede a focar nos canais mais relevantes. Cada bloco MBConv é definido por parâmetros como o fator de expansão (que determina o quanto os canais de entrada são expandidos), o tamanho do kernel da convolução, o stride (passo da convolução) e a razão SE (squeeze-and-excitation), que controla a intensidade da recalibração dos canais. O Head é composto por um bloco final de convolução, seguido por uma camada de pooling médio global (global average pooling). Por fim, há uma camada totalmente conectada (fully connected) com uma função de ativação softmax, que realiza a classificação final.

Fonte: https://www.geeksforgeeks.org/efficientnet-architecture/

In [ ]:
device = "GPU" if len(tf.config.list_physical_devices('GPU')) > 0 else "CPU"
print(f"Usando: {device}")

In [ ]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(image_size, 3))
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid') 
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=32,
    verbose=1
)

In [ ]:
y_pred_prob = model.predict(X_test).ravel()
y_pred = (y_pred_prob > 0.5).astype(int)

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(5, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Real', 'Fake'], yticklabels=['Real', 'Fake'])
plt.xlabel("Predito")
plt.ylabel("Real")
plt.title("Matriz de Confusão")
plt.show()

print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred, target_names=["Real", "Fake"]))

roc_auc = roc_auc_score(y_test, y_pred_prob)
print(f"AUC (ROC): {roc_auc:.4f}")

plt.figure(figsize=(8, 4))
plt.plot(history.history['accuracy'], label='Treino')
plt.plot(history.history['val_accuracy'], label='Validação')
plt.title("Curva de Aprendizado - Acurácia")
plt.xlabel("Época")
plt.ylabel("Acurácia")
plt.legend()
plt.grid()
plt.show()

plt.figure(figsize=(8, 4))
plt.plot(history.history['loss'], label='Treino')
plt.plot(history.history['val_loss'], label='Validação')
plt.title("Curva de Aprendizado - Perda")
plt.xlabel("Época")
plt.ylabel("Perda (Loss)")
plt.legend()
plt.grid()
plt.show()
